In [6]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# MNISTデータセットをロード
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像を正規化
X_train = X_train.reshape((60000, 28, 28, 1)) / 255.0
X_test = X_test.reshape((10000, 28, 28, 1)) / 255.0

# ネットワークの層数を変化させる
for layers in [1, 2, 3, 5]:
    # モデルを構築
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    for _ in range(layers):
        model.add(Dense(128, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    # データ量を変化させる
    for n in [100, 1000, 10000, 60000]:
        # モデルを学習
        model.fit(X_train[:n], y_train[:n], epochs=5, batch_size=128, verbose=0)
        
        # テストデータで評価
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"層数: {layers}, データ量: {n}, 精度: {accuracy:.3f}")

2024-12-12 22:41:32.688064: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 22:41:32.721758: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 22:41:32.722526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 22:41:33.447654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


11490434/11490434 [==============================] - 1s 0us/step


2024-12-12 22:41:37.776926: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-12 22:41:37.779504: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


層数: 1, データ量: 100, 精度: 0.391
層数: 1, データ量: 1000, 精度: 0.843
層数: 1, データ量: 10000, 精度: 0.938
層数: 1, データ量: 60000, 精度: 0.974
層数: 2, データ量: 100, 精度: 0.403
層数: 2, データ量: 1000, 精度: 0.857
層数: 2, データ量: 10000, 精度: 0.942
層数: 2, データ量: 60000, 精度: 0.977
層数: 3, データ量: 100, 精度: 0.313
層数: 3, データ量: 1000, 精度: 0.848
層数: 3, データ量: 10000, 精度: 0.945
層数: 3, データ量: 60000, 精度: 0.976
層数: 5, データ量: 100, 精度: 0.452
層数: 5, データ量: 1000, 精度: 0.819
層数: 5, データ量: 10000, 精度: 0.948
層数: 5, データ量: 60000, 精度: 0.976


In [23]:
import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow import keras
from tensorflow.keras import layers

# 1. データの生成 (サンプルデータ)
num_samples = 1000
num_features = 220

# 入力データ: 220次元の正規分布に従う乱数を1000サンプル生成
X = np.random.randn(num_samples, num_features)
# 目的変数: 2次元の正規分布に従う乱数を1000サンプル生成
# 例として、出力1は入力の最初の100個の和 + ノイズ, 出力2は入力の最後の120個の和 + ノイズ
pprint.pprint(len(X[0]))

220


In [27]:
y = np.zeros((num_samples, 2))
y[:, 0] = X[:, :100].sum(axis=1) + np.random.randn(num_samples) * 5  # ノイズを加える
y[:, 1] = X[:, 100:].sum(axis=1) + np.random.randn(num_samples) * 5 # ノイズを加える

# 2. データの分割 (訓練データとテストデータ)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train[0]),len(y_test[0]))

220 2


In [ ]:


# 3. モデルの構築
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=(num_features,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(2),  # 出力層は2次元、活性化関数なし（線形関数）
    ]
)

# 4. モデルのコンパイル
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# 5. モデルの学習
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

# 6. モデルの評価
y_pred = model.predict(X_test)

# 評価指標の計算
mse = mean_squared_error(y_test, y_pred
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

# 7. (オプション) 予測結果の確認
print("最初の5つの予測結果:")
for i in range(5):
    print(f"予測: {y_pred[i]}, 実際: {y_test[i]}")

Epoch 1/100
23/23 [==============================] - 1s 7ms/step - loss: 137.4556 - mae: 9.4120 - val_loss: 139.6441 - val_mae: 9.2800
Epoch 2/100
23/23 [==============================] - 0s 3ms/step - loss: 131.8898 - mae: 9.2091 - val_loss: 136.7129 - val_mae: 9.1761
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 123.3533 - mae: 8.8871 - val_loss: 127.9511 - val_mae: 8.8572
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 105.9692 - mae: 8.1969 - val_loss: 110.9703 - val_mae: 8.2180
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 78.6001 - mae: 6.9863 - val_loss: 86.9058 - val_mae: 7.2537
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 48.9708 - mae: 5.4505 - val_loss: 65.9575 - val_mae: 6.3060
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 28.7822 - mae: 4.2176 - val_loss: 53.3590 - val_mae: 5.7010
Epoch 8/100
23/23 [==============================] - 0s 5ms/s